In [2]:
%cd ym

/home/jupyter/AoU_DRC_WGS_LongReads_PacBio/edit/ym


In [7]:
# download merged callsets
!gsutil cp gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/T2T/v1/concat_annotated.sens_09.vcf.gz .
!gsutil cp gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/T2T/v1/concat_annotated.sens_07.vcf.gz .

Copying gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/T2T/v1/concat_annotated.sens_09.vcf.gz...
- [1 files][941.8 MiB/941.8 MiB]   43.1 MiB/s                                   
Operation completed over 1 objects/941.8 MiB.                                    
Copying gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/T2T/v1/concat_annotated.sens_07.vcf.gz...
| [1 files][556.5 MiB/556.5 MiB]   43.0 MiB/s                                   
Operation completed over 1 objects/556.5 MiB.                                    


In [9]:
!mv concat_annotated.sens_07.vcf.gz concat_annotated.sens_07.T2T.vcf.gz
!mv concat_annotated.sens_09.vcf.gz concat_annotated.sens_09.T2T.vcf.gz

In [10]:
# download merged callsets
!gsutil cp gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/GRCh38/v1/concat_annotated.sens_09.vcf.gz .
!gsutil cp gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/GRCh38/v1/concat_annotated.sens_07.vcf.gz .

Copying gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/GRCh38/v1/concat_annotated.sens_09.vcf.gz...
- [1 files][732.3 MiB/732.3 MiB]   42.4 MiB/s                                   
Operation completed over 1 objects/732.3 MiB.                                    
Copying gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/GRCh38/v1/concat_annotated.sens_07.vcf.gz...
/ [1 files][451.2 MiB/451.2 MiB]   45.8 MiB/s                                   
Operation completed over 1 objects/451.2 MiB.                                    


In [4]:
# UNK-->INV for LR file (fixing known issue in truvari anno svinfo)
from pysam import VariantFile

def unk_to_inv(vcf_in_filename, vcf_out_filename):
    vcf_in = VariantFile(vcf_in_filename)
    vcf_out = VariantFile(vcf_out_filename, 'w', header=vcf_in.header)

    for rec in vcf_in.fetch():
        if rec.info['SVTYPE']=='UNK':
            rec.info['SVTYPE']='INV'
        vcf_out.write(rec) # write every variant

In [5]:
# run UNK-->INV
# this is a little silly, I could just do the replace afterwards - this takes SO LONG with the huge VCF
unk_to_inv("concat_annotated.sens_09.vcf.gz","concat_annotated.sens_09.unk_to_inv.vcf")
!bcftools view -Oz concat_annotated.sens_09.unk_to_inv.vcf > concat_annotated.sens_09.unk_to_inv.vcf.gz
!tabix concat_annotated.sens_09.unk_to_inv.vcf.gz
!rm concat_annotated.sens_09.unk_to_inv.vcf

ERROR! Session/line number was not unique in database. History logging moved to new session 37


[E::idx_find_and_load] Could not retrieve index file for 'concat_annotated.sens_09.vcf.gz'


In [7]:
!bcftools query -f "%SVTYPE\n" concat_annotated.sens_09.unk_to_inv.vcf.gz | sort | uniq -c |awk 'OFS="\t" {print $2,$1}' > tpr09_whole_cohort_count_by_svtype

In [15]:
!bcftools query -f "%SVTYPE\n" concat_annotated.sens_07.vcf.gz | sort | uniq -c |awk 'OFS="\t" {print $2,$1}' > tpr07_whole_cohort_count_by_svtype

In [16]:
!ls -lht *whole_cohort_count_by_svtype

-rw-rw-r-- 1 jupyter users 30 Jun  2 02:09 tpr07_whole_cohort_count_by_svtype
-rw-rw-r-- 1 jupyter users 32 Jun  2 02:08 tpr09_T2T_whole_cohort_count_by_svtype
-rw-rw-r-- 1 jupyter users 30 Jun  2 01:59 tpr07_T2T_whole_cohort_count_by_svtype
-rw-rw-r-- 1 jupyter users 31 May 30 19:15 tpr09_whole_cohort_count_by_svtype


In [8]:
!head tpr09_whole_cohort_count_by_svtype

DEL	279385
INS	934491
INV	1684


In [9]:
!bcftools view -Oz concat_annotated.sens_09.unk_to_inv.vcf.gz chr1:1000000-5000000 > test_small.vcf.gz

In [20]:
# per-sample counts by svtype
from pysam import VariantFile
import sys

def calls_per_sample(vcf_in_filename, outfile):
    vcf_in = VariantFile(vcf_in_filename)
    outfile = open(outfile,'w')
    
    samples = list(vcf_in.header.samples)
    
    sample_counts = {}
    for sample in samples:
        sample_counts[sample] = {'total':0, 'INS':0,'DEL':0,'UNK':0}
    
    chrom=None
    for rec in vcf_in.fetch():
        if rec.chrom != chrom: # just a status update because this is so slow
            chrom = rec.chrom
            sys.stderr.write("%s\n" % chrom)
        svtype = rec.info['SVTYPE']
        for i in range(0,len(samples)):
            sample = samples[i]
            if 1 in rec.samples[sample]['GT']:
                sample_counts[sample]['total']+=1
                sample_counts[sample][svtype] +=1

    for sample in sample_counts:
        outfile.write("%s\t%d\t%d\t%d\t%d\n" % (sample, sample_counts[sample]['total'], 
                              sample_counts[sample]['INS'],
                              sample_counts[sample]['DEL'],
                              sample_counts[sample]['UNK']))
    outfile.close()

In [ ]:
calls_per_sample("concat_annotated.sens_09.T2T.vcf.gz", "counts_per_genome_by_svtype_09_T2T")

[E::idx_find_and_load] Could not retrieve index file for 'concat_annotated.sens_09.T2T.vcf.gz'
chr1
chr2
chr3
chr4


In [59]:
!bcftools view -h concat_annotated.sens_09.unk_to_inv.vcf.gz|tail -n1|cut -f10- | sed 's/\t/\n/g' |head -n1027 > samples

In [60]:
# do the python code above but in a bcftools one-liner - I think this will be faster? actually maybe the other way is faster
!rm counts_per_genome_by_svtype
!while read sample; do echo -n "$sample"$'\t' >> counts_per_genome_by_svtype; bcftools query -i 'GT=="alt"' -f '%SVTYPE\n' -s $sample concat_annotated.sens_09.unk_to_inv.vcf.gz|sort|uniq -c |awk -v RS="INV" '{print $1+$3+$5"\t"$1"\t"$3"\t"$5}' | head -n1 >> counts_per_genome_by_svtype; done < samples 

^C


In [55]:
#!echo "10580 DEL" > tmp
#!echo "15515 INS" >> tmp
#!echo "38 INV" >> tmp
#!echo "1050 DEL" >> tmp
#!echo "1515 INS" >> tmp
#!echo "3 INV" >> tmp
!sample=105;echo -n "$sample"$'\t'
!awk -v RS="INV" '{print $1+$3+$5"\t"$1"\t"$3"\t"$5}' tmp

105	26133	10580	15515	38
2568	1050	1515	3
0			


In [13]:
!head test_small_counts_per_genome_by_svtype

1000151	180	99	81	0
1000513	175	103	72	0
1000920	181	121	60	0
1001399	186	114	72	0
1001980	150	103	47	0
1002322	187	115	72	0
1002826	170	106	64	0
1004266	167	111	56	0
1005038	154	94	60	0
1005444	177	112	65	0


In [6]:
!bcftools query -f '%SVTYPE\t%SVLEN\n' concat_annotated.sens_09.unk_to_inv.vcf.gz > tpr09_whole_cohort_length_by_svtype